# EE 344 — Assignment 4: Fake News Classification

In this assignment, you will classify news articles as **fake vs real** using **text features**.
Your tasks for this assignment are as follows:

1. Learn how to extract text features by vectorizing textual inputs using **CountVectorizer (Bag-of-Words)**.
2. Implement **7 classifiers**: Logistic Regression, Perceptron, Linear SVM (LinearSVC), Multinomial Naive Bayes, KNN, Decision Tree, and Random Forest.
3. Evaluate **train and test** performance using **accuracy, precision, recall, and F1-score**.
4. Provide brief answers to discussion questions about (i) the text feature extraction method you implemented and (ii) the effect of using two different KNN distance choices (**Euclidean vs cosine**).


## Submission guidelines
- Complete all **[TODO]** blocks in this notebook.
- Push the finished notebook to your GitHub repository.
- Submit the GitHub link on the Canvas submission page.


**Dataset source (for reference only):**  
Do **not** download data from the link below. Use the provided `evaluation.csv` file that comes with this assignment.
#### https://www.kaggle.com/datasets/aadyasingh55/fake-news-classification



## Setup
Run the next cell to import libraries and define helper functions.


In [20]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.linear_model import LogisticRegression, Perceptron
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Reproducibility
RANDOM_STATE = 42

def metrics(y_true, y_pred):
    """Return (accuracy, precision, recall, f1)."""
    acc = accuracy_score(y_true, y_pred)

    # Use 'binary' for binary classification, otherwise fallback to macro.
    avg = "binary" if len(np.unique(y_true)) == 2 else "macro"

    prec, rec, f1, _ = precision_recall_fscore_support(
        y_true, y_pred, average=avg, zero_division=0
    )
    return acc, prec, rec, f1


## Load data

Put the dataset file in the same folder as this notebook (recommended), or provide an absolute path.

This dataset uses **semicolon-separated** fields and can contain extra semicolons inside the text.
So we use a custom loader that safely reconstructs the text column.


In [21]:
# === Data path ===
# If your file has a different name/path, update it here.
DATA_PATH = "evaluation.csv"   # <-- change if needed

def load_semicolon_dataset(path):
    """
    Handles lines like:
    ;title;text;label
    0;some title;some text that may contain ; ; ; ;0
    """
    rows = []
    with open(path, "r", encoding="utf-8", errors="replace") as f:
        _ = f.readline()  # header
        for line in f:
            line = line.rstrip("\n")
            if not line:
                continue
            parts = line.split(";")
            if len(parts) < 4:
                continue

            idx = parts[0]
            title = parts[1]
            label = parts[-1]
            text = ";".join(parts[2:-1])  # re-join any extra ';' inside text
            rows.append((idx, title, text, label))

    df = pd.DataFrame(rows, columns=["id", "title", "text", "label"])
    df["label"] = pd.to_numeric(df["label"], errors="coerce")
    df = df.dropna(subset=["label"]).reset_index(drop=True)
    df["label"] = df["label"].astype(int)
    return df

df = load_semicolon_dataset(DATA_PATH)
print("Dataset:", df.shape)
print("Label distribution:\n", df["label"].value_counts())

# Combine title + text into one string per document
docs = (df["title"].fillna("") + " " + df["text"].fillna("")).astype(str).tolist()
y = df["label"].values


Dataset: (7815, 4)
Label distribution:
 label
1    4185
0    3630
Name: count, dtype: int64


## Train/test split

We keep a standard **80/20** split with stratification (preserves label ratio).


In [22]:
X_train_text, X_test_text, y_train, y_test = train_test_split(
    docs, y,
    test_size=0.20,
    random_state=RANDOM_STATE,
    stratify=y
)

print("Train docs:", len(X_train_text), "Test docs:", len(X_test_text))


Train docs: 6252 Test docs: 1563


## Case Study: Bag-of-Words Features (CountVectorizer)

We need to convert text into numeric features before we can train ML models.

**CountVectorizer** builds a vocabulary from the **training set** and represents each document as a vector of **counts** (one entry per vocabulary term).

We will use:
$$
\texttt{CountVectorizer(}
\texttt{lowercase=True, stop_words="english", ngram_range=(1,2),}
$$
$$
\texttt{ min_df=2, max_df=0.9, max_features=10000)}
$$

**What each setting means (briefly):**
- `lowercase=True`: convert text to lowercase before building features.
- `stop_words="english"`: remove a predefined list of common English words.
- `ngram_range=(1,2)`: allow 1-word features and 2-word features (bigrams).
- `min_df=2`: keep a term only if it appears in at least 2 training documents.
- `max_df=0.9`: drop a term if it appears in more than 90% of training documents.
- `max_features=10000`: cap the vocabulary size at 10,000 terms (after filtering).

### Tiny example (just to see what it does)

We will build features from 3 short documents and look at the counts.


In [23]:
# CountVectorizer docs (read this once before TODO 1):
# https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html


In [24]:
toy_docs = [
    "The FAKE news spreads fast",
    "Fake news spreads",
    "Real news spreads",
]

toy_vec = CountVectorizer(
    lowercase=True,
    stop_words="english",
    ngram_range=(1, 2),
    min_df=2
)

toy_X = toy_vec.fit_transform(toy_docs)

print("Toy vocab:", list(toy_vec.get_feature_names_out()))
print("Toy counts (rows = docs):\n", toy_X.toarray())


Toy vocab: ['fake', 'fake news', 'news', 'news spreads', 'spreads']
Toy counts (rows = docs):
 [[1 1 1 1 1]
 [1 1 1 1 1]
 [0 0 1 1 1]]


## Build Bag-of-Words features

Goal:
1. Create the `CountVectorizer` using the exact settings below.
2. Fit on the training text only.
3. Transform both train and test text into sparse Bag-of-Words features.

Notes:
- `fit_transform` on train, then `transform` on test.
- The output is a **sparse matrix** (CSR). That is normal for text features.


In [25]:
# --- Bag-of-Words settings ---
MAX_FEATURES = 10000
NGRAM_RANGE = (1, 2)

## [ TODO 1 ]
# 1) Create `vectorizer` using CountVectorizer with:
#    lowercase=True
#    stop_words="english"
#    ngram_range=NGRAM_RANGE
#    min_df=2
#    max_df=0.9
#    max_features=MAX_FEATURES
#
# 2) Fit the vectorizer on the training text, then use it to transform:
#    - the training text into BoW features
#    - the test text into BoW features
#
# (Reminder: fit on train only; do NOT fit on test.)
#
# Print the BoW shapes.
vectorizer = CountVectorizer(
    lowercase=True,
    stop_words='english',
    ngram_range=NGRAM_RANGE,
    min_df=2,
    max_df=0.9,
    max_features=MAX_FEATURES
)

X_train_bow = vectorizer.fit_transform(X_train_text)
X_test_bow = vectorizer.transform(X_test_text)

print("BoW shapes:", X_train_bow.shape, X_test_bow.shape)



BoW shapes: (6252, 10000) (1563, 10000)


## Models

Create **7 classifiers** using the exact hyperparameters below.

**Important:** For KNN in this notebook, start with **Euclidean distance**.

Models to implement:
- Logistic Regression: `solver="saga"`, `max_iter=2000`, `n_jobs=-1`, `random_state=42`
- Perceptron: `max_iter=1000`, `tol=1e-3`, `random_state=42`
- SVM (LinearSVC): `random_state=42`
- Naive Bayes (MultinomialNB): `alpha=1.0`
- KNN (Euclidean): `n_neighbors=7`, `metric="euclidean"`, `n_jobs=-1`
- Decision Tree: `max_depth=40`, `random_state=42`
- Random Forest: `n_estimators=300`, `random_state=42`, `n_jobs=-1`

Put them in a dictionary named `models`.


In [29]:
## [ TODO 2 ]
# Build the `models` dictionary using the exact parameters above.

models = {
    'Logistic Regression': LogisticRegression(solver='saga', max_iter=2000, n_jobs=-1, random_state=42),
    'Perceptron': Perceptron(max_iter=1000, tol=1e-3, random_state=42),
    'SVM': LinearSVC(random_state=42),
    'Naive Bayes': MultinomialNB(alpha=1.0),
    'KNN (Euclidean)': KNeighborsClassifier(n_neighbors=7, metric='euclidean', n_jobs=-1),
    'KNN (Cosine)': KNeighborsClassifier(n_neighbors=7, metric='cosine', algorithm='brute', n_jobs=-1),
    'Decision Tree': DecisionTreeClassifier(max_depth=40, random_state=42),
    'Random Forest': RandomForestClassifier(n_estimators=300, random_state=42, n_jobs=-1)
}

models



{'Logistic Regression': LogisticRegression(max_iter=2000, n_jobs=-1, random_state=42, solver='saga'),
 'Perceptron': Perceptron(random_state=42),
 'SVM': LinearSVC(random_state=42),
 'Naive Bayes': MultinomialNB(),
 'KNN (Euclidean)': KNeighborsClassifier(metric='euclidean', n_jobs=-1, n_neighbors=7),
 'KNN (Cosine)': KNeighborsClassifier(algorithm='brute', metric='cosine', n_jobs=-1,
                      n_neighbors=7),
 'Decision Tree': DecisionTreeClassifier(max_depth=40, random_state=42),
 'Random Forest': RandomForestClassifier(n_estimators=300, n_jobs=-1, random_state=42)}

## Train + evaluate

We will evaluate each model on:
- **Training set**
- **Test set**

Metrics:
- Accuracy
- Precision
- Recall
- F1

We will print a table sorted by **Test F1**.


In [30]:
## [ TODO 3 ]
# Write a loop that:
# 1) fits each model on X_train_bow, y_train
# 2) predicts on train and test
# 3) computes (acc, prec, rec, f1) using metrics(...)
# 4) stores results in a list
# 5) prints a DataFrame sorted by Test F1 (descending)
#
# Use the exact column names below.

results = []

for name, model in models.items():
    model.fit(X_train_bow, y_train)

    y_pred_train = model.predict(X_train_bow)
    y_pred_test = model.predict(X_test_bow)

    results.append([
        name,
        *metrics(y_train, y_pred_train),
        *metrics(y_test, y_pred_test)
    ])


cols = [
    "Model",
    "Train Acc", "Train Prec", "Train Rec", "Train F1",
    "Test Acc", "Test Prec", "Test Rec", "Test F1",
]

out = pd.DataFrame(results, columns=cols).sort_values("Test F1", ascending=False).reset_index(drop=True)

pd.set_option("display.max_colwidth", 80)
print("\n=== Results (sorted by Test F1) ===")
print(out.to_string(index=False, formatters={
    "Train Acc": "{:.4f}".format,
    "Train Prec": "{:.4f}".format,
    "Train Rec": "{:.4f}".format,
    "Train F1": "{:.4f}".format,
    "Test Acc": "{:.4f}".format,
    "Test Prec": "{:.4f}".format,
    "Test Rec": "{:.4f}".format,
    "Test F1": "{:.4f}".format,
}))

# raise NotImplementedError("Complete TODO 3 above.")


/home/tdokkad/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(



=== Results (sorted by Test F1) ===
              Model Train Acc Train Prec Train Rec Train F1 Test Acc Test Prec Test Rec Test F1
      Random Forest    1.0000     1.0000    1.0000   1.0000   0.9949    0.9976   0.9928  0.9952
      Decision Tree    1.0000     1.0000    1.0000   1.0000   0.9904    0.9940   0.9881  0.9910
Logistic Regression    0.9989     0.9997    0.9982   0.9990   0.9872    0.9869   0.9892  0.9881
                SVM    1.0000     1.0000    1.0000   1.0000   0.9853    0.9857   0.9869  0.9863
         Perceptron    1.0000     1.0000    1.0000   1.0000   0.9808    0.9821   0.9821  0.9821
        Naive Bayes    0.9624     0.9684    0.9612   0.9648   0.9610    0.9630   0.9642  0.9636
       KNN (Cosine)    0.9128     0.8717    0.9818   0.9234   0.8791    0.8299   0.9737  0.8961
    KNN (Euclidean)    0.7844     0.8906    0.6810   0.7718   0.7364    0.8571   0.6093  0.7123


## Cosine distance for KNN

With Bag-of-Words, each document becomes a long vector of word counts (mostly zeros).  
To compare two documents, we need a way to measure how “close” two vectors are.

Two common choices:

- **Euclidean distance**: straight-line distance between two vectors.
- **Cosine distance**: based on the angle between two vectors (uses cosine similarity under the hood).

In scikit-learn, KNN uses a **distance**. Cosine distance is:
$$
d_{\text{cosine}}(x, z) \;=\; 1 - \cos(x, z)
\;=\; 1 - \frac{x^\top z}{\|x\|_2 \,\|z\|_2}
$$

(where $\cos(x,z)$ is cosine similarity).

### Tiny numeric example (no text, just vectors)

Let:
- $x = [1, 1]$
- $z_1 = [2, 2]$  (same direction as $x$, just “bigger”)
- $z_2 = [2, 0]$  (different direction)

**Euclidean distances**
$$
\|x - z_1\|_2 = \sqrt{(1-2)^2 + (1-2)^2} = \sqrt{2}
$$
$$
\|x - z_2\|_2 = \sqrt{(1-2)^2 + (1-0)^2} = \sqrt{2}
$$
So Euclidean says $z_1$ and $z_2$ are equally far from $x$ here.

**Cosine distances**
$$
\cos(x, z_1) = \frac{1\cdot 2 + 1\cdot 2}{\sqrt{2}\cdot \sqrt{8}} = 1
\Rightarrow d_{\text{cosine}}(x, z_1)=0
$$
$$
\cos(x, z_2) = \frac{1\cdot 2 + 1\cdot 0}{\sqrt{2}\cdot 2} \approx 0.707
\Rightarrow d_{\text{cosine}}(x, z_2)\approx 0.293
$$
So cosine says $z_1$ is closer to $x$ than $z_2$.

### What you will do

Keep everything the same, but change your KNN metric from `"euclidean"` to `"cosine"`, then re-run your evaluation and compare results.


In [28]:
# Tip: For cosine distance, brute-force search is commonly used.
# Example (do not run until TODO 2/3 are done):
#
# knn_cos = KNeighborsClassifier(
#     n_neighbors=7,
#     metric="cosine",
#     algorithm="brute",
#     n_jobs=-1
# )


## Discussion questions (answer in your own words)

Write short answers below (2–5 sentences each is enough).

### Question A
In your own words, what is the added value of allowing 2-word sequences (bigrams) in `ngram_range`?

### Question B
In your own words, why might someone choose to set both `min_df` and `max_df` when building the vocabulary?

### Question C

After you run KNN with **Euclidean** and then with **Cosine** distance:

- Do you observe any difference in results?
- If yes, why do you think the difference happens (your intuition)?



**Your answers:**

- **A:**  
  This allows the model to recognize common phrases (like 'fake news' in the toy example) that are common across the texts analyzed. This is particularly helpful for phrases that have an idiomatic meaning aside from that of the component words.

- **B:**  
  These parameters do different things; min_df is for removing rare words that occur too infrequently to make a good generalization about what their impact is while max_df is for removing common words (especially lexically weak ones like 'the', 'and', 'is', &c). Either due to sampling error/lack of info or excessive prevalance, both categories fail to be useful diagnostics. Pruning them helps save compute and/or make space for more helpful predictors.

- **C:**  
  The KNN model using cosine distance performs notably better than the Euclidean difference. It makes sense that cosine distance would work well here because cosine distance, having a hard maximum, can be 'maxed out' easily even in a sparse feature set, and the angle of BoW vectors (what words are featured) is probably a lot more relevant than their magnitudes (how many are featured). At the same time, the BoW features contain no negative values, so that values can approach maximum distance if they have differences in many dimensions but can't hit it directly. I am not entirely sure why the Euclidean distance performs so badly here, however.
